In [31]:
import sys
import pandas as pd
import sklearn.cross_validation as cv

from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

#path to the data
DATA_PATH = '../../data/train.csv'

def main(argv=None):
    #parameter k for k-fold cv
    if argv is None:
        argv = sys.argv

    processing()
    
def processing():
    data = pd.read_csv(DATA_PATH, index_col=0)

    print(type(data))
#    for i in data.iteritems():
#        print(i)
    
if __name__ == '__main__':
    main()
    

<class 'pandas.core.frame.DataFrame'>
